In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import json
import numpy as np
from intent_exs import IntentEXS
from pyserini.search.lucene import LuceneSearcher  

index_path = '/home/wang/xaiss/datasets/scidocs/corpus_index'
query = 'what is explainable AI?'

searcher = LuceneSearcher(index_path)   # load a searcher from pre-computed index.

hits = searcher.search(query)
# Print the first 10 hits:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')
    print()
# extract the retrieved doc ids and doc contents.
doc_ids = [hit.docid for hit in hits]
docs = dict([(hit.docid, json.loads(searcher.doc(hit.docid).raw())['contents']) for hit in hits])

# Load a reranking model
from beir.reranking.models import CrossEncoder
model = 'cross-encoder/ms-marco-electra-base'
reranker = CrossEncoder(model)

# build query-doc pair for reranking model as input.
sentence_pairs = []
for doc_id in doc_ids:
    doc_text = docs[doc_id]
    sentence_pairs.append([query, doc_text])
rerank_scores = reranker.predict(sentence_pairs, batch_size=1)

# show reranked docs.
reranked_docids = np.array(doc_ids)[np.argsort(rerank_scores)[::-1]]
for i, doc_id in enumerate(reranked_docids):
    print('doc_id =', doc_id)
    #if i < 1:
    print('contents =', docs[doc_id])


    
# build corpus for IntentEXS explain function
corpus = {'query': query,
        'scores': dict([(doc_id, score) for doc_id, score in zip(doc_ids, rerank_scores)]),
        'docs': docs
}
params = {'top_idf': 10, 'topk': 5, 'max_pair': 100, 'max_intent': 10, 'style': 'random'}

 1 8394eca39f743cf4cd098fb92cc4718d34be1d04 9.11290

 2 d8ed29985999ca9642e7940fbf7484f544df54e5 7.17950

 3 1e21c514f89375098dec5b947aa5f6bcdd0377c5 6.05780

 4 6258323b0ddedd0892febb36c1772a10820e0b0c 6.01710

 5 4f6b7f8daa322801887b2ef8c2c14788e607e3b8 5.96180

 6 20c7139595570f080fc85a054c84262d11a488bd 5.95310

 7 34c84bc46dbac879d82b675d2adf2b7f1911d387 5.75280

 8 66ea9389b7ecd7f8070cddc8c6c3ecbf301e3577 5.73630

 9 24ff5027e7042aeead47ef3071f1a023243078bb 5.60510

10 511921e775ab05a1ab0770a63e57c93da51c8526 5.58010



Batches: 100%|██████████| 10/10 [00:00<00:00, 44.99it/s]

doc_id = 8394eca39f743cf4cd098fb92cc4718d34be1d04
contents = Stakeholders in Explainable AI There is general consensus that it is important for artificial intelligence (AI) and machine learning systems to be explainable and/or interpretable. However, there is no general consensus over what is meant by ‘explainable’ and ‘interpretable’. In this paper, we argue that this lack of consensus is due to there being several distinct stakeholder communities. We note that, while the concerns of the individual communities are broadly compatible, they are not identical, which gives rise to different intents and requirements for explainability/interpretability. We use the software engineering distinction between validation and verification, and the epistemological distinctions between knowns/unknowns, to tease apart the concerns of the stakeholder communities and highlight the areas where their foci overlap or diverge. It is not the purpose of the authors of this paper to ‘take sides’ — we count ou

In [24]:
# Init the IntentEXS object.
Intent = IntentEXS(reranker, index_path, 'bm25')
expansion = Intent.explain(corpus, params)
expansion

Batches: 100%|██████████| 1/1 [00:00<00:00, 36.29it/s]


Picked!
Picked!
Picked!
Picked!
Picked!
Picked!
Picked!


['explain', 'range', 'use', 'key', 'architecture', 'stakeholder', 'summarize']

Dataset: scidocs
query: what is CRONA?
content: ['provide', 'kind', 'what', 'guide', 'article']
content + title: ['what','employ','cognitive','business','describe', 'accurate', 'become','application']
title: ['what', 'make', 'ontology', 'function', 'about', 'from', 'marketing', 'base']